## Простенький код на умение писать регулярные выражения для работы с датафреймами: задача - из строки с наименованием, огрн и инн разбить информацию в отдельные столбцы

In [9]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_excel('Разбить на столбцы.xlsx')

In [3]:
data

,Заемщик (идентификационные сведения)
0,Акционерное общество «Щекиноазот»\n(АО «Щекино...
1,"Акционерное общество ""Щекиноазот"" ОГРН: 102710..."
2,"Акционерное общество ""Щекиноазот"" ОГРН: 102710..."
3,"Акционерное общество ""Метафракс Кемикалс"" (АО ..."
4,"ООО ""Акрил Салават""\nИНН 0266040210\nОГРН 1120..."
...,...
72,"АО ""Соликамскбумпром""\nОГРН 1025901975802\nИНН..."
73,"Акционерное общество ""Соликамскбумпром"" \n(АО ..."
74,"Акционерное общество ""Балтийская целлюлоза"" (А..."
75,"Общество с ограниченной ответственностью ""Борс..."


## Перебираю все кейсы, чтобы правильно вытащить данные из строки и записать их в новые столбцы

In [4]:
def extract_ogrn(row):
    match = re.search(r'ОГРН[:\s]?\s*(\d+)', row)
    return match.group(1) if match else None

def extract_inn(row):
    match = re.search(r'ИНН[:\s]?\s*(\d+)', row)
    return match.group(1) if match else None

def extract_name(row, ogrn, inn):
    if ogrn:
        row = re.sub(r'ОГРН[:\s]?\s*' + ogrn, '', row)
    if inn:
        row = re.sub(r'ИНН[:\s]?\s*' + inn, '', row)
    return row.strip()

In [5]:
data['ОГРН'] = data['Заемщик (идентификационные сведения)'].apply(extract_ogrn)
data['ИНН'] = data['Заемщик (идентификационные сведения)'].apply(extract_inn)
data['Наименование'] = data.apply(lambda row: extract_name(row['Заемщик (идентификационные сведения)'], row['ОГРН'], row['ИНН']), axis=1)

In [6]:
data

,Заемщик (идентификационные сведения),ОГРН,ИНН,Наименование
0,Акционерное общество «Щекиноазот»\n(АО «Щекино...,1027100507015,7118004789,Акционерное общество «Щекиноазот»\n(АО «Щекино...
1,"Акционерное общество ""Щекиноазот"" ОГРН: 102710...",1027100507015,7118004789,"Акционерное общество ""Щекиноазот"""
2,"Акционерное общество ""Щекиноазот"" ОГРН: 102710...",1027100507015,7118004789,"Акционерное общество ""Щекиноазот"""
3,"Акционерное общество ""Метафракс Кемикалс"" (АО ...",1025901777571,5913001268,"Акционерное общество ""Метафракс Кемикалс"" (АО ..."
4,"ООО ""Акрил Салават""\nИНН 0266040210\nОГРН 1120...",1120280051990,0266040210,"ООО ""Акрил Салават"""
...,...,...,...,...
72,"АО ""Соликамскбумпром""\nОГРН 1025901975802\nИНН...",1025901975802,5919470121,"АО ""Соликамскбумпром"""
73,"Акционерное общество ""Соликамскбумпром"" \n(АО ...",1025901975802,5919470121,"Акционерное общество ""Соликамскбумпром"" \n(АО ..."
74,"Акционерное общество ""Балтийская целлюлоза"" (А...",1037804004204,7802005550,"Акционерное общество ""Балтийская целлюлоза"" (А..."
75,"Общество с ограниченной ответственностью ""Борс...",1177746590427,9718070427,"Общество с ограниченной ответственностью ""Борс..."


In [7]:
columns_to_drop = [col for col in data.columns if col not in ['ОГРН', 'ИНН' , 'Наименование']]
data = data.drop(columns=columns_to_drop)

In [8]:
data

,ОГРН,ИНН,Наименование
0,1027100507015,7118004789,Акционерное общество «Щекиноазот»\n(АО «Щекино...
1,1027100507015,7118004789,"Акционерное общество ""Щекиноазот"""
2,1027100507015,7118004789,"Акционерное общество ""Щекиноазот"""
3,1025901777571,5913001268,"Акционерное общество ""Метафракс Кемикалс"" (АО ..."
4,1120280051990,0266040210,"ООО ""Акрил Салават"""
...,...,...,...
72,1025901975802,5919470121,"АО ""Соликамскбумпром"""
73,1025901975802,5919470121,"Акционерное общество ""Соликамскбумпром"" \n(АО ..."
74,1037804004204,7802005550,"Акционерное общество ""Балтийская целлюлоза"" (А..."
75,1177746590427,9718070427,"Общество с ограниченной ответственностью ""Борс..."
